<img src="https://www.strath.ac.uk/media/1newwebsite/webteam/logos/xUoS_Logo_Horizontal.png.pagespeed.ic.M6gv_BmDx1.png" width="350" align="left" style="left">

# 1. RFSoC Device Overview <a class="anchor" id="rfsoc_overview"></a>

This notebook will be focused around the  architecture of the RFSoC hardware.  

## Table of Contents

* [1. RFSoC Introduction](01_introduction.ipynb)
    * [1.1 High Level Overview](#high_level_overview)
    * [1.2 Notable Features](#notable_features)
        * [1.2.1 Single Chip Integration](#single_chip)
        * [1.2.2 High Sample Rate](#sample_rate)
        * [1.2.3 Multiple Channels](#multi_channel)
    * [1.3 PS Architecture](#ps_arch)
    * [1.4 PL Architecture](#pl_arch)
    * [1.5 Data Converters](#data_converters)
    * [1.6 RF-ADC Hierarchy](#rfadc_hier)
        * [1.6.1 RF-ADC Tiles](#rfadc_tiles)
        * [1.6.2 Dual RF-ADC Tiles](#dual_rfadc_tiles)
        * [1.6.3 Quad RF-ADC Tiles](#quad_rfadc_tiles)
        * [1.6.4 DDC: Digital I/Q Mixer](#ddc)
        * [1.6.5 I/Q Mixer Modes](#IQ_mixer)
        * [1.6.6 DDC: Programmable Decimator](#prog_decimator)
        * [1.6.7 Active Gain Control](#agc)
    * [1.7 RF-DAC Hierarchy](#rfdac_hier)
        * [1.7.1 RF-DAC Programmable Interpolator](#rfdac_interp)
    * [1.8 SD-FEC Blocks](#sd_fec)  
    * [1.9 Conclusion](#conclusion)
 

## 1.1 High Level Overview <a class="anchor" id="high_level_overview"></a>

The RFSoC is a Zynq System on Chip (SoC) targeted specifically at RF applications. It features a Processing System (PS), Programmable Logic (PL) and specialist RF resources. A high level overview of the RFSoC can be seen in [Figure 1](#fig-1).

<a class="anchor" id="fig-1"></a>
<figure>
<img src='./images/RFSoC_overview.png' height='30%' width='30%'/>
    <figcaption><b>Figure 1: High Level Overveiw of the RFSoC hardware</b></figcaption>
</figure>

The specialist RF resources included in the RFSoC are:

- **Transmit Blocks**: 8 or 16 very high rate RF Digital to Analogue Converters (RF-DACs). These blocks also include Digital Up Converters (DUCs) for increasing the sampling rate and performing modulation. 


- **Receive blocks**: 8 or 16 very high rate RF Analogue-to-Digital (RF-ADCs). These blocks feature Digital Down Converters (DDCs) for demodulating the received signal and then reducing the sampling rate.


- **Soft Decision Forward Error Correction (SD-FEC) blocks**: 8 blocks for FEC encoding and/or decoding. These blocks support FEC coding schemes that are commonly used in 4G and 5G mobile standards.

There are also hardened resources for high speed wired communications, in the form of GTY Transceiver blocks. These blocks are commonly required for backhaul in wireless networks, namely CPRI (Common Public Radio Interface) and GigE (Gigabit Ethernet).

A general overview of the RFSoC device family can be seen from the Xilinx product page:

https://www.xilinx.com/products/silicon-devices/soc/rfsoc.html#ProductTable

## 1.2 Notable Features <a class="anchor" id="notable_features"></a>

The RFSoC device has a number of notable features which separate it from a traditional radio transceiver.

**1.2.1 Single Chip Integration** <a class="anchor" id="single_chip"></a>

The RFSoC integrates DACs and ADCs into the same chip as the processing resources needed to implement a radio transceiver.  

Other solutions require at least 2 chips e.g. a Zynq/Zynq MPSoC for signal processing paired with a front-end radio transceiver for modulating and demodulating the signal to and from its target RF Frequency. This system can be seen in [Figure 2](#fig-2).

<a class="anchor" id="fig-2"></a>
<figure>
<img src='./images/two_chip_solution.png' height='30%' width='30%'/>
    <figcaption><b>Figure 2: A traditional two chip solution utalising a Zynq MPSoC and a front-end radio transceiver</b></figcaption>
</figure>

As a result, the RF interface ports of the RFSoC development board can be connected directly to analogue components to build a complete radio system. 

**1.2.2 High Sample Rate** <a class="anchor" id="sample_rate"></a>

The RF-ADCs and RF-DACs are both capable of sampling at multiple GHz, providing a very wide RF bandwidth. The RFSoC can directly digitise an RF band from 0 to 2 GHz. 

**1.2.3 Multiple Channels** <a class="anchor" id="multi_channel"></a>

Each RFSoC device has multiple channels (8 or 16) of transmit and receive functionality. These have a number of uses, including Multiple Input Multiple Output (MIMO) systems and Beamforming/Beamsteering introducing spatial diversity to improve the throughput achievable over a link using multiple transmitters and multiple receivers involved, each with a different position in space.

Aside from these multi-antenna systems, the 8 or 16 channels of the RFSoC can also be used to support multiple different transmit and receive paths of the same or different radio standards on the same chip. This compares favourably to a conventional solution where several chips would be required. 

## 1.3 PS Architecture <a class="anchor" id="ps_arch"></a>

The RFSoC's Processing System (PS) includes several different types of processing resources, as seen in [Figure 3](#fig-3). These can be used to implement the software-based aspects of a transmission/receiver system. These resources include:

- **External Interfaces**: local IoT devices, SDR backhaul, networked system control.


- **Platform Management and security functions**: system booting, power management, physical security, SDR comms security.


- **Dual-core Arm Cortex-R5 real-time processor**: low level, deterministic SDR system control and orchestration.


- **PS local memories, external memory control**: Software programs, SDR instructions and data


- **Quad-core Arm Cortex-A53 applications processor**: running operating system, SDR software applications

<a class="anchor" id="fig-3"></a>
<figure>
<img src='./images/PS_Diagram.png' height='25%' width='25%'/>
    <figcaption><b>Figure 3: High level overview of the RFSoC PS</b></figcaption>
</figure>

The four Arm Cortex-A53 processors are capable of running a fully featured Operating System and software applications. In the examples in this course the PYNQ software framework is deployed on the applications processor. More details on PYNQ can be found here: http://www.pynq.io/.

## 1.4 PL Architecture <a class="anchor" id="pl_arch"></a> 

The Programmable Logic of the RFSoC is equivalent to that of an FPGA. Most devices in the RFSoC family provide a standard set of resources, including:

- High speed, wired interfaces (GTY transceivers, PCIe, 150G Interlaken, 100G Ethernet)


- Configurable Logic Blocks (930K logic cells)


- DSP Slices (4272)


- Block RAM (38 Mb)


- Ultra RAM (22.5 Mb)


AXI PS-to-PL interconnections enable control and data to pass quickly between the software and hardware systems.

The PL is where the hardware design will be created. The parallel, programmable nature of the PL is ideal for implementing Digital Signal Processing (DSP) for the physical layer of transmitter and receivers. Hardware subsystems can be designed for flexible operation, programmed from software.

## 1.5 Data Converters <a class="anchor" id="data_converter"></a>

There are two types of RF Data Converters on the RFSoC:

**Transmit side**: An RF-DAC and Digital Up-Converter (DUC). The Interpolators, NCO and sampling rate are programmable. The RF-DAC can be seen in [Figure 4](#fig-4).

<a class="anchor" id="fig-4"></a>
<figure>
<img src='./images/DAC_block.png' height='50%' width='50%'/>
    <figcaption><b>Figure 4: Overview of RF-DAC</b></figcaption>
</figure>

**Receive side**: An RF-ADC and Digital Down-Converter (DDC), likewise programmable. The RF-ADC can be seen in [Figure 5](#fig-5).

<a class="anchor" id="fig-5"></a>
<figure>
<img src='./images/ADC_block.png' height='50%' width='50%'/>
    <figcaption><b>Figure 5: Overview of RF-ADC</b></figcaption>
</figure>

A notable benefit to the RFSoC is the highly programmable nature of the RF Data Converters:

- The sampling rates for the RF-DACs and RF-ADCs are programmable, with maximum and minimum values depending on the RFSoC Generation. For Gen 1, the RF-DACs can operate at sample rates of up to 6.554 Gsps, and the RF-ADCs can operate at up to 4.096 Gsps.


- The NCO frequency is programmable. When the standard operating mode is used, it represents the RF frequency that the baseband signal is modulated to, and demodulated from.


- The Interpolators and decimators can be programmed to undertake rate changes of 1 (no rate change), 2, 4 and 8, respectively. Further rate interpolation or decimation can be implemented in the PL.

### 1.6 RF-ADC Hierarchy <a class="anchor" id="rfadc_hier"></a>

The architecture and hierarchy of the RF-ADCs and RF-DACs are key to the RFSoC design and as such will now be considered in more detail, begining with the RF-ADCs. 

**1.6.1 RF-ADC Tiles** <a class="anchor" id="rfadc_tiles"></a>

RFSoC devices have either 8 or 16 RF-ADCs, grouped into tiles alongside mixers and decimators.

There are two different tile configurations, shown in [Figure 6](#fig-6):

- Dual RF-ADC tiles: 2x RF-ADCs, 2x mixers, 2x decimators
- Quad RF-ADC tiles: 4x RF-ADCs, 4x mixers, 4x decimators

<a class="anchor" id="fig-6"></a>
<figure>
<img src='./images/Simple_ADC_Tiles.png' height='50%' width='50%'/>
    <figcaption><b>Figure 6: ADC Tile Configurations</b></figcaption>
</figure>

Each RF-ADC tile includes its own Phase Locked Loop (PLL) for managing the sampling clock applied to the RF-ADCs. 

The RFSoC's RF-ADCs comprise multiple interleaved sub-ADCs, all sampling at a lower rate. In combination, these can achieve a sampling rate of up to 5 Gsps.

**1.6.2 Dual RF-ADC Tile** <a class="anchor" id="dual_rfadc_tiles"></a>

A dual RF-ADC tile supports two RF input channels, as can be seen in [Figure 7](#fig-7):

<a class="anchor" id="fig-7"></a>
<figure>
<img src='./images/Dual_ADC_Tile.png' height='50%' width='50%'/>
    <figcaption><b>Figure 7: Dual ADC Tile</b></figcaption>
</figure>

RF-ADCs can operate in two modes:

- **Real-to-Complex**: An RF channel is input to the RF Data Converter as a real signal, and is thereafter mixed with the output of a complex Numerically Controlled Oscillator (NCO) to form a complex (I/Q) output. The upper and lower halves of the tile can each handle one input RF signal in this manner, and therefore the Dual RF-ADC tile can support two RF channels.


- **Complex-to-Complex**: An RF channel is input to the RF Data Converter as a complex signal (I/Q), and is then mixed using a complex oscillator to form a complex output (also I/Q). In this configuration, both RF-ADCs are required - one for the analogue I input and another for the analogue Q output. Consequently, in this mode, a Dual RF-ADC tile can only support one RF channel.

The user has a number of programmable options when operating Dual RF-ADC Tiles:

- Operating mode (Real-to-Complex or Complex-to-Complex)


- RF-ADC sampling frequency


- Mixer mode (course or fine)


- Mixer NCO frequency


- Decimation ratio (1x, 2x, 4x, 8x) - note that 1x simply involves bypassing the decimator


Signals are demodulated using the complex NCO and decimated using the integrated decimator. Both of the RF-ADCs within the tile share the same clocking infrastructure, enabling synchronous sampling.

**1.6.3 Quad RF-ADC Tile** <a class="anchor" id="quad_rfadc_tiles"></a>

A Quad RF-ADC tile comprises four RF-ADCs, arranged in two pairs. This can be seen in [Figure 8](#fig-8).

<a class="anchor" id="fig-8"></a>
<figure>
<img src='./images/Quad_ADC_Tile.png' height='50%' width='50%'/>
    <figcaption><b>Figure 8: Quad ADC Tile</b></figcaption>
</figure>

A quad RF-ADC tile is equivalent to two Dual RF-ADC tiles. One advantage that it has over the Dual RF-ADC is its ability to support multiple bands (i.e. spectrally separated signals). The Quad can support up to four bands, the Dual, two.

Multiple bands can be received simultaneously by routing the sampled RF input signal to more than one different mixing and decimation stage within the tile. As there are two mixers and two decimators with the Dual RF-ADC it can support two bands simultaneously, whereas the Quad RF-ADC can support up to four bands using four mixers and four decimators.

**1.6.4 DDC: Digital I/Q Mixer** <a class="anchor" id="ddc"></a>

The digital I/Q mixer multiplies the incoming signal with sine and cosine waves, generated by an NCO. This has the effect of shifting the input signal up or down in frequency. This works to demodulate the received signal, shifting it to a baseband frequency. 

Learn more about modulation and demodulation from this [notebook](./Modulation_Demodulation.ipynb).

**1.6.5 I/Q Mixer Modes** <a class="anchor" id="IQ_mixer"></a>

The I/Q Mixer can operate in 3 modes: **Coarse**, **Fine** and **Bypass**. The RFSoC I/Q Mixer circuitry can be seen in [Figure 9](#fig-9).

<a class="anchor" id="fig-9"></a>
<figure>
<img src='./images/IQ_mixer.png' height='50%' width='50%'/>
    <figcaption><b>Figure 9: Overview of I/Q Mixer circuitry showing 3 oeprating modes</b></figcaption>
</figure>

The **Coarse Mixer** component implements both the **Coarse** mode and the **Bypass** mode.

- In **Coarse** mode, the mixer is able to demodulate using only a very restricted set of frequencies, however it is able to operate in this mode with lower power than in Fine mode. This is because sine and cosine waves at these frequencies comprise a very limited set of samples.


- In **Bypass** mode, the signal path simply bypasses the I/Q mixing stage, via the coarse stage, and does not undergo any modulation/demodulation.

Frequencies of $\frac{f_{s}}{4}$ and $-\frac{f_{s}}{4}$ are useful for modulating/demodulating to a quarter of the sampling frequency with absolutely minimal cost. The NCO must generate only four samples per sine wave cycle: 0, +1, 0, -1. It is trivial to store these values and to multiply another signal by them for modulation/demodulating. 

Multiplying by 0, +1 and -1 is trivial, and therefore the modulator or demodulator architecture can be simplified to a counter and some simple combinatorial logic, as shown in [Figure 10](#fig-10).

<a class="anchor" id="fig-10"></a>
<figure>
<img src='./images/course_IQ.png' height='25%' width='25%'/>
    <figcaption><b>Figure 10: Course I/Q Mixer implemented using a counter and combinatorial logic</b></figcaption>
</figure>

This same principle can be implemented to demodulate with $\frac{f_{s}}{2}$ as well.

The **Fine Mixer** component implements **Fine** mode. This can be seen in [Figure 11](#fig-11).

- In **Fine** mode, the mixer uses an NCO, which can generate any arbitrary frequency between $-\frac{f_{s}}{2}$ and $\frac{f_{s}}{2}$. This is done by setting a step size input to the phase accumulator (automatically calculated in the design tools), the lookup table size and the desired frequency. Optional additional parameters can be set, including a phase adjustment and added dithering.

<a class="anchor" id="fig-11"></a>
<figure>
<img src='./images/fine_mixer.png' height='50%' width='50%'/>
    <figcaption><b>Figure 11: Fine Mixer Circuitry</b></figcaption>
</figure>

**1.6.6 DDC: Programmable Decimator** <a class="anchor" id="prog_decimator"></a>

The decimator can perform rate reduction by a factor of: 1x, 2x, 4x, or 8x.

Decimation is achieved by a set of half-band filters: FIR0, FIR1, and FIR2. These low pass filters each decimate by a factor of 2, and are cascaded together to form  the selection of outputs shown in [Figure 12](#fig-12):

<a class="anchor" id="fig-12"></a>
<figure>
<img src='./images/ProgrammableDecimator.png' height='50%' width='50%'/>
    <figcaption><b>Figure 12: Programmable Decimator</b></figcaption>
</figure>

**1.6.7 Active Gain Control** <a class="anchor" id="agc"></a>
       
Active Gain Control (AGC) is an adaptive method for adjusting the gain applied to the input signal of an ADC, in order to scale the voltage to match the full scale of the ADC.

In RFSoC, the AGC algorithm is user-designed in the PL, and controls an external Variable Gain Amplifier (VGA). An input signal can be amplified to fit the range of the ADC or attenuated if the input signal is too large.

The inputs to the AGC algorithm are control signals generated by the RF-ADC block, which indicate when the input signal level exceeds amplitude thresholds. The algorithm should then compute the neccessary gain digitally and send a control signal to the VGA to adjust its level.

### 1.7 RF-DAC Hierarchy <a class="anchor" id="rfdac_hier"></a>

The RF-DACs are grouped in a similar fashion to RF-ADCs. The main difference is that RF-DACs are available only in Quad tiles, i.e. there are always four RF-DAC blocks per tile. The components available in an Rf-DAC Quad Core Tile can be seen in [Figure 13](#fig-13)

<a class="anchor" id="fig-13"></a>
<figure>
<img src='./images/Simple_DAC_Tiles.png' height='25%' width='25%'/>
    <figcaption><b>Figure 13: DAC Tile Configuration, Only Available as Quad Tiles</b></figcaption>
</figure>

RF-DAC blocks can be configured individually, to produce real output signals, or as a pair, to produce complex output signals.

Each RF-DAC block contains a programmable interpolator, an I/Q mixer, and the RF-DAC data converter. 

**1.7.1 RF-DAC Programmable Interpolator** <a class="anchor" id="rfdac_interp"></a>

The programmable interpolator supports ratios of 1x, 2x, 4x, and 8x. An interpolation by 1 means that the filters are bypassed.

The programmable interpolators are composed of three halfband filters, FIR0, FIR1 and FIR2. The path taken through the filters depends on the chosen interpolation ratios. This structure is very similar to that used in the RF-ADC's programmable decimator, but notice that the order of the filters differs. The specification of the individual filters is the same as in the RF-ADC. The RF-DAC interpolator can be seen in [Figure 14](#fig-14).

<a class="anchor" id="fig-14"></a>
<figure>
<img src='./images/ProgrammableInterp.png' height='50%' width='50%'/>
    <figcaption><b>Figure 14: RF-DAC Programmable Interpolator</b></figcaption>
</figure>

## 1.8 SD-FEC Blocks <a class="anchor" id="sd_fec"></a>

FEC coding and decoding are computationally intensive processes, particularly for the codes adopted in 4G and 5G radio systems. The Soft Decision (SD)-FEC core in the RFSoC supports the following modes of operation:

- Low Density Parity Check (**LDPC**) coding


- **LDPC** decoding


- **Turbo** decoding

Both LDPC and Turbo codes are high performance coding schemes that have been adopted in modern communication standards. Turbo codes are used in 3G and 4G Long Term Evolution (LTE). LDPC codes are used in 5G, providing even better error correction performance.

Selected RFSoC chips provide 8 hard SD-FEC cores. Soft SD-FEC cores can also be implemented in the PL if desired.

Learn more about SD-FEC from this [notebook](https://github.com/Xilinx/PYNQ_RFSOC_Workshop/blob/master/notebooks/03_pynq_and_sd_fec.ipynb).

## 1.9 Conclusion <a class="anchor" id="conclusion"></a>

This notebook has provided an overview of the hardware blocks which comprise the RFSoC. We have explored a high level overview of the full system as well as the PS and PL of the RFSoC before taking a closer look at each block.

You should now be aware of:

- **High Level Overview** - the RFSoC is comprised of Programmable Logic, a Processing System and several special blocks, including Transmit, Receive and SD-FEC blocks
- **Notable Features** - High Sampling Rate, Single Chip Solution, Multiple Channels
- **PS Architecture** - Acts as the 'brain' of the RFSoC. Contains: allows for external interfaces, security functions and memory control
- **PL Architecture** - Equivalent to an FPGA. Contains: high speed wired interfaces, Configurable Logic Blocks, DSP Slices, Block RAM, Ultra RAM.
- **Data Converters** - RFSoC is special due to inbuilt RF-ADC bad RF-DAC Data Converters.
- **RF-ADC Hierarchy** - Arranged in Dual or Quad tiles. DDC allows for decimation and a digital I/Q mixer with multiple modes. Active Gain Control is also available
- **RF-DAC Hierarchy** - Arranged in Quad Tiles. Programmable Interpolators and Digital I/Q mixer available. 
- **SD-FEC Blocks** - Supports LDPC coding and decoding as well as turbo decoding

[⬅️ Previous Notebook](01_introduction.ipynb) | | [Next Notebook ➡️](03_rfsoc_for_sdr.ipynb)

----
----